In [1]:
!pip install transformers torch numpy opensearch-py bert-score sacrebleu rouge-score langchain langchain-community langchainhub langchain-chroma bs4 langchain-huggingface
!pip install -qU langchain-openai
!python -m spacy download de_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [1]:
# Standard Library
import math
import os
import re
import time

# Third-Party Libraries
import bs4
import numpy as np
import requests
import sacrebleu
import spacy
import torch
from rouge_score import rouge_scorer
from bert_score import score
from opensearchpy import OpenSearch
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline

# LangChain Libraries
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.docstore.document import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# OpenAI
from openai import OpenAI

In [2]:
# Connect to OpenSearch instance
dbclient = OpenSearch(
    hosts=[{'host': 'opensearch-ds.ifi.uni-heidelberg.de', 'port': 443}],
    http_auth=('ryousaf', 'i5am3SHER4locked'),  # Replace with your credentials
    use_ssl=True,
    verify_certs=False,
    ssl_show_warn=False
)

# Load the spacy model for keyword extraction
nlp = spacy.load("de_core_news_lg")

# Define stopwords
STOPWORDS = set([
    "mg", "symptome", "behandlung", "folgen", "mangel"
])

def extract_keyword_ner(query):
    doc = nlp(query)

    # Extract named entities
    keywords = [ent.text for ent in doc.ents]

    # If no named entities found, fall back to extracting nouns and proper nouns
    if not keywords:
        keywords = [token.text for token in doc if token.pos_ in ['NOUN', 'PROPN']]

    # Filter out stopwords (ensure case-insensitive comparison)
    filtered_keywords = [keyword for keyword in keywords if keyword.lower() not in STOPWORDS]

    # Return the list of filtered keywords
    print(filtered_keywords)
    return filtered_keywords

# Search for all documents related to query
def find_title(index, keyword):

    concatenated_text = ""
    for word in keyword:
        # Define the search query
        query = {
            "query": {
                "match": {
                    "title": word
                }
            }
        }

        try:
            # Execute the search query
            response = dbclient.search(index=index, body=query)

            # Check if we got any hits
            if response['hits']['total']['value'] > 0:
                for hit in response['hits']['hits']:
                    text = hit['_source']['text']
                    concatenated_text += text + "\n"  # Add each hit's text to the concatenated_text
                    print(f"Title: {hit['_source']['title']}")
                    #print(f"Text: {text}")
                    print(f"URL: {hit['_source']['url']}")
                    print("-" * 80)
            else:
                print("Keine Ergebnisse für ", word, " gefunden")
        except Exception as e:
            print(f"Fehler beim Ausführen der Suchanfrage: {e}")

    return concatenated_text

In [3]:
def prepare_docs(query):

    # Retrieve relevant docs
    keywords = extract_keyword_ner(query)
    retrieved_docs = find_title("kic_apothekenumschau_articles", keywords)

    # Load retrieved content
    docs = [Document(page_content=retrieved_docs, metadata={"source": "local"})]

    # Chunk the retrieved content
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)

    # Define the embedding model
    embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

    # Index the chunked content
    vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
    retriever = vectorstore.as_retriever()

    return retriever


def execute_query(retriever, system_prompt, query):

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    response = rag_chain.invoke({"input": query})
    return response["answer"]

In [4]:
# Setting LLM model
os.environ["OPENAI_API_KEY"] = "sk-proj-nG7Aj0B6nhRht4EwEYB7T3BlbkFJzr4aIVHGH57ztjG4LUZY"
llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
# Standard prompt as control group
std_prompt = (
    "Du bist ein intelligenter Agent. Dir wird eine Frage gestellt und Du erhälst die relevanten Informationen. Deine Aufgabe besteht darin, die Frage mit den bereitgestellten Informationen zu beantworten."
    "\n\n"
    "{context}"
)

# Simple prompt: Answer in plain language
simple_prompt = (
    "Du bist ein intelligenter Agent. Dir wird eine Frage gestellt und Du erhälst die relevanten Informationen. Deine Aufgabe besteht darin, die Frage in leichter Sprache mit den bereitgestellten Informationen zu beantworten."
    "\n\n"
    "{context}"
)

# Rule-based prompt with simple set of rules
simple_rules = (
    "Du bist ein intelligenter Agent. Dir wird eine Frage gestellt und Du erhälst die relevanten Informationen. Deine Aufgabe besteht darin, die Frage in leichter Sprache basierend auf den Regeln zu beantworten und die bereitgestellten Informationen zu verwenden."
    '''Regeln: Wörter:
      • Verwende einfache Wörter.
      • Vermeide Fachbegriffe und erkläre, wenn nötig.
      • Verwende dieselben Wörter für dieselben Dinge.
      • Verwende keine Abkürzungen.
      • Verwende Verben, keine Verbalsubstantive.
      • Verwende aktive Wörter.

      Zahlen und Sonderzeichen:
      • Verwende Zahlen, wie wir sie kennen.
      • Vermeide Sonderzeichen oder erkläre sie nicht.

      Sätze:
      • Schreibe kurze Sätze (Subjekt – Verb – Objekt).
      • Verwende keine Nebensätze.
      • Sprich den Leser informell an.'''
    "\n\n"
    "{context}"
)

# Example-based prompt with short example text
simple_example = (
    "Du bist ein intelligenter Agent. Dir wird eine Frage gestellt und Du erhälst die relevanten Informationen. Deine Aufgabe besteht darin, die Frage in leichter Sprache wie in dem Beispiel zu beantworten und die bereitgestellten Informationen zu verwenden."
    '''Beispiel: Mit unseren Texten in Einfacher Sprache wollen wir vielen Menschen diese
    Informationen zugänglich machen. Auf unserer Internetseite sollen sich
    alle Leser zum Thema Gesundheit informieren können."
    **Worum geht es in den Texten?**
    Auf der Internetseite der Apotheken Umschau gibt es sehr viele
    Informationen zum Thema Gesundheit. Zur Zeit gibt es etwa 130 Texte in
    Einfacher Sprache. Diese Texte informieren über ganz unterschiedliche
    Themen.
    Es gibt Texte zu vielen verschiedenen Krankheiten, zum Beispiel:
    - Adipositas
    - Gürtelrose
    - Weitsichtigkeit
    Es gibt Texte zu vielen Medikamenten und Heilpflanzen, zum Beispiel:
    - Ibuprofen
    - Arnika
    - Kurkuma
    Wie enstehen die Texte?
    Ein Übersetzer der Forschungsstelle Leichte Sprache
    übersetzt den Text. Dann prüfen zwei andere Übersetzer: Ist die
    Übersetzung verständlich geschrieben?
    Dann prüfen medizinische Fachleute vom Wort & Bild Verlag: Sind die Informationen in der Übersetzung richtig?
    Der Text ist sprachlich leicht verständlich geschrieben?
    Und die Informationen im Text sind richtig und vollständig?
    Erst dann kommen die Texte auf die Internetseite.'''
    "\n\n"
    "{context}"
)

## Example-based and rule-based prompt
rules_example = (
    "Du bist ein intelligenter Agent. Dir wird eine Frage gestellt und Du erhälst die relevanten Informationen. Deine Aufgabe besteht darin, die Frage in leichter Sprache basierend auf den Regeln und wie in dem Beispiel zu beantworten und die bereitgestellten Informationen zu verwenden."
    '''Regeln: Wörter:
      • Verwende einfache Wörter.
      • Vermeide Fachbegriffe und erkläre, wenn nötig.
      • Verwende dieselben Wörter für dieselben Dinge.
      • Verwende keine Abkürzungen.
      • Verwende Verben, keine Verbalsubstantive.
      • Verwende aktive Wörter.

      Zahlen und Sonderzeichen:
      • Verwende Zahlen, wie wir sie kennen.
      • Vermeide Sonderzeichen oder erkläre sie nicht.

      Sätze:
      • Schreibe kurze Sätze (Subjekt – Verb – Objekt).
      • Verwende keine Nebensätze.
      • Sprich den Leser informell an.'''
    '''Beispiel: Mit unseren Texten in Einfacher Sprache wollen wir vielen Menschen diese
    Informationen zugänglich machen. Auf unserer Internetseite sollen sich
    alle Leser zum Thema Gesundheit informieren können."
    **Worum geht es in den Texten?**
    Auf der Internetseite der Apotheken Umschau gibt es sehr viele
    Informationen zum Thema Gesundheit. Zur Zeit gibt es etwa 130 Texte in
    Einfacher Sprache. Diese Texte informieren über ganz unterschiedliche
    Themen.
    Es gibt Texte zu vielen verschiedenen Krankheiten, zum Beispiel:
    - Adipositas
    - Gürtelrose
    - Weitsichtigkeit
    Es gibt Texte zu vielen Medikamenten und Heilpflanzen, zum Beispiel:
    - Ibuprofen
    - Arnika
    - Kurkuma
    Wie enstehen die Texte?
    Ein Übersetzer der Forschungsstelle Leichte Sprache
    übersetzt den Text. Dann prüfen zwei andere Übersetzer: Ist die
    Übersetzung verständlich geschrieben?
    Dann prüfen medizinische Fachleute vom Wort & Bild Verlag: Sind die Informationen in der Übersetzung richtig?
    Der Text ist sprachlich leicht verständlich geschrieben?
    Und die Informationen im Text sind richtig und vollständig?
    Erst dann kommen die Texte auf die Internetseite.'''
    "\n\n"
    "{context}"
)

In [6]:
query = "Wer ist besonders von Vitamin-D Mangel betroffen?"

docs = prepare_docs(query)

std_answer = execute_query(docs, std_prompt, query)
print("std answer: ", std_answer)

simple_answer = execute_query(docs, simple_prompt, query)
print("simple answer: ", simple_answer)

rules_answer = execute_query(docs, simple_rules, query)
print("rules answer: ", rules_answer)

example_answer = execute_query(docs, simple_example, query)
print("example answer: ", example_answer)

rules_example_answer = execute_query(docs, rules_example, query)
print("rules + example answer: ", rules_example_answer)

['Vitamin-D']
Title: Vitamin D
URL: https://www.apotheken-umschau.de/mein-koerper/haut-und-haare/vitamin-d-711907.html
--------------------------------------------------------------------------------
Title: Vitamin D und Covid-19
URL: https://www.apotheken-umschau.de/krankheiten-symptome/infektionskrankheiten/coronavirus/vitamin-d-und-covid-19-768277.html
--------------------------------------------------------------------------------
Title: So wirkt Colecalciferol (Vitamin D)
URL: https://www.apotheken-umschau.de/medikamente/wirkstoffe/colecalciferol-so-wirkt-vitamin-d-1005313.html
--------------------------------------------------------------------------------
Title: Wie viel Vitamin D brauchen Kinder?
URL: https://www.apotheken-umschau.de/familie/kindergesundheit/vitamin-d-fuer-kinder-848211.html
--------------------------------------------------------------------------------
Title: Wie sinnvoll sind Vitamin-D-Tabletten?
URL: https://www.apotheken-umschau.de/krankheiten-symptome/gel

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


std answer:  Besonders von Vitamin-D-Mangel betroffen sind Säuglinge und Kinder, da ein zu niedriger Vitamin-D-Spiegel bei ihnen zu Rachitis führen kann, was bedeutet, dass die Knochen nicht ausreichend mineralisiert werden und weich bleiben. Auch ältere Menschen ab 65 Jahren sind anfällig, da bei ihnen nicht mehr so viel Vitamin D über die Haut hergestellt wird. Darüber hinaus können jüngere Menschen, die überwiegend im Büro sitzen und im Winter nur wenig Tageslicht abbekommen, ebenfalls an Vitamin-D-Mangel leiden.
simple answer:  Besonders von Vitamin-D-Mangel betroffen sind:

1. **Säuglinge und Kinder**: Sie können an Rachitis leiden, wenn ihr Vitamin-D-Spiegel zu niedrig ist.
2. **Ältere Menschen ab 65 Jahren**: Sie stellen weniger Vitamin D über die Haut her und können daher einen Mangel haben.
3. **Jüngere Menschen, die viel drinnen sind**: Wenn sie wenig Sonnenlicht bekommen, können sie ebenfalls an Vitamin-D-Mangel leiden, besonders im Winter.

Es ist wichtig, auf ausreichende 

In [7]:
# BERTScore
def calculate_bertscore(refs, hyps):
    P, R, F1 = score(refs, hyps, lang='de')
    return P.mean().item(), R.mean().item(), F1.mean().item()

# BLEU Score
def calculate_bleu(refs, hyps):
    bleu = sacrebleu.corpus_bleu(hyps, [refs])
    return bleu.score

# ROUGE Score
def calculate_rouge(refs, hyps):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(refs, hyps)
    return {key: scores[key].fmeasure for key in scores}

def evaluate(reference, generated):

    # Calculate the BERT score
    P, R, F1 = calculate_bertscore([reference], [generated])
    print(f'BERTScore - Precision: {P:.4f}, Recall: {R:.4f}, F1 Score: {F1:.4f}')

    # Calculate BLEU score
    bleu = calculate_bleu([reference], [generated])
    print(f'BLEU Score: {bleu:.4f}')

    # Calculate ROUGE score
    rouge = calculate_rouge(reference, generated)
    for key in rouge:
        print(f'ROUGE {key} Score: {rouge[key]:.4f}')
    print("\n")

In [8]:
evaluate(std_answer, simple_answer)
evaluate(std_answer, rules_answer)
evaluate(std_answer, example_answer)
evaluate(std_answer, rules_example_answer)

BERTScore - Precision: 0.7721, Recall: 0.7250, F1 Score: 0.7478
BLEU Score: 15.3444
ROUGE rouge1 Score: 0.5402
ROUGE rouge2 Score: 0.3488
ROUGE rougeL Score: 0.4368


BERTScore - Precision: 0.7960, Recall: 0.8295, F1 Score: 0.8124
BLEU Score: 16.7695
ROUGE rouge1 Score: 0.5034
ROUGE rouge2 Score: 0.2483
ROUGE rougeL Score: 0.4626


BERTScore - Precision: 0.7336, Recall: 0.7241, F1 Score: 0.7288
BLEU Score: 13.4861
ROUGE rouge1 Score: 0.4564
ROUGE rouge2 Score: 0.2449
ROUGE rougeL Score: 0.3624


BERTScore - Precision: 0.7876, Recall: 0.7419, F1 Score: 0.7641
BLEU Score: 16.1248
ROUGE rouge1 Score: 0.6000
ROUGE rouge2 Score: 0.3131
ROUGE rougeL Score: 0.4500




In [9]:
def evaluate_llm(text):
    messages = [
        SystemMessage(content="Du bist ein intelligenter Agent. Du bekommst einen Text und sollst für jede der folgenden Kriterien kurz mit Ja oder Nein beantworten, ob sie erfüllt werden. Gebe am Ende eine korrigierte Version des Textes."
        '''Kriterien:
          1. Alle Sätze sind Hauptsätze der Form "Subjekt - Verb - Objekt".
          2. Es gibt keine Nebensätze.
          3. Alle Fachbegriffe, die nicht alltagsgebrauchlich sind, werden erklärt.
          4. Der Leser wird angesprochen.
          5. Es wird kein Genitiv verwendet.
          6. Es wird kein Konjunktiv verwendet.
          7. Es gibt keine Abkürzungen.
        '''),
        HumanMessage(content=text),
    ]

    response = llm(messages)
    return response

# Use the function with your string
response = evaluate_llm(rules_example_answer)
print(response.content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


1. Nein
2. Ja
3. Ja
4. Ja
5. Ja
6. Ja
7. Ja

Korrigierte Version des Textes:
Besonders Menschen mit Vitamin-D-Mangel sind betroffen. 

Säuglinge und Kinder haben ein Risiko. Sie leiden an Rachitis. Rachitis ist eine Krankheit. Sie macht die Knochen weich und verformt sie. 

Erwachsene haben auch ein Risiko. Ein Mangel führt zu Osteomalazie. Osteomalazie bedeutet, dass die Knochen weich werden. Osteoporose kann auch auftreten. Osteoporose ist ein verstärkter Knochenabbau. 

Ältere Menschen ab 65 Jahren haben ein Risiko. Sie bilden weniger Vitamin D über die Haut. 

Junge Menschen sind auch betroffen. Besonders Menschen, die viel im Büro sitzen, haben ein Risiko. Sie bekommen im Winter wenig Tageslicht. 

Es ist wichtig, den Vitamin-D-Spiegel zu prüfen. Prüfen Sie den Spiegel, wenn Sie sich unsicher fühlen.
